## Setup

In [ ]:
%run ../utils/utils.ipynb

from copy import deepcopy
from itertools import product

from sklearn.preprocessing import scale, robust_scale, normalize

In [ ]:
# Used to simulate no scaling or normalizing of the data
def no_transform(data):
    return data

In [ ]:
## Constants
# datasets is a list of all potential datasets created in data_processing.ipynb
kernels = ['Matern12',
           'Matern12_Linear',
           'Matern32',
           'Matern32_Linear',
           'Matern52',
           'Matern52_Linear',
           'RBF',
           'RBF_Linear']

scaling_funcs = [scale, robust_scale, no_transform]

# We experimented with normalizing the data as well, however we found that there lacked 
# a good theoretical reason for why it works and so dropped it
# normalizing_funcs = [normalize, no_transform]

In [ ]:
# Emulates sklearn's grid_search over the parameter space with modifications for our use case
# by simply searching over the potential datasets, kernels, scaling and normalizing options
def grid_search(datasets, kernels, scaling):
    results = {}
    
    # Product returns a generator over the cartesian product of input iterables
    for dataset, kernel, scaler in product(datasets, kernels, scaling):
        # It is necessary to deepcopy the dataset in order to perform the scaling changes
        # or else scaling and normalizing changes would be performed on the original object
        dataset = deepcopy(dataset)
        name = "_".join([kernel, 
                         dataset.agg_level,
                         dataset.value_type,
                         dataset.binner,
                         scaler.__name__])
        print('Testing:', name)
        start = time.time()
        dataset.x = scaler(dataset.x)
        results[name] = cv(dataset, kernel)
        end = time.time()
        print('Time elapsed (sec):', end - start)
    
    pickle_results(results, 'grid_search')
    print('Results saved to disk')

## Run through - full grid

In [ ]:
grid_search(datasets, kernels, scaling_funcs)

## Scratchpad